Amirkabir University of Technology

Applied Linear Algebra

Dr Amirmazlaghani

By: Gholamreza Dar 400131018

March 2022


## Imports and inits

In [106]:
import numpy as np
import pandas as pd
from IPython.display import display, Math

from fractions import Fraction
from functools import reduce
from math import lcm


In [107]:
M = 0 # number of rows(elements)
N = 0 # number of columns(compounds) + 1(b=0)
elements = [] # Chemical Elements
compounds = [] # Chemical Compounds
coeff_matrix = None # Coefficient Matrix

## Helper Functions

### def compound_to_vector()

In [108]:
def compound_to_vector(compound, elements):
    """This function converts a compound to a vector.
    Args:
        compound (str): Chemical compound as an array of elements.
        for example ['C2', 'H6']
        elements (list): Chemical elements as an array.
        for example ['C', 'H', 'O']

    Returns:
        vector (list): count of atoms per element as a vector of size len(elements).
        
    example:
        compound = 'C2H4'
        elements = ['C', 'H', 'O']
        returns:
        [2,4,0]
    """
    vector = np.zeros(len(elements))
    for element in compound:
        if len(element) == 1:
            vector[elements.index(element)] = 1
        else:
            vector[elements.index(element[0])] = element[1:]
    return vector

In [109]:
# testing our function
compound_to_vector(
    ['H20', 'O6'], 
    elements="C H O".split())

array([ 0., 20.,  6.])

### Index finders 
(non zero row and column finders)

In [110]:
def get_left_most_non_zero_column_index(matrix, start_row_index):
    is_col_non_zero_list = [matrix[start_row_index:,i].any() for i in range(matrix.shape[1])]
    if np.any(is_col_non_zero_list):
        return np.argmax(is_col_non_zero_list)
    else:
        return -1

def get_top_most_non_zero_row_index(matrix, start_row_index, pivot_column_index):
    is_row_non_zero_list = [matrix[i,pivot_column_index].any() for i in range(matrix.shape[0])]

    # Ignore the rows before i
    for i in range(start_row_index):
        is_row_non_zero_list[i] = False

    if np.any(is_row_non_zero_list):
        return np.argmax(is_row_non_zero_list)
    else:
        return -1
    
def get_top_most_non_zero_row_index_thats_not_current_row(matrix, current_row_index, start_row_index, pivot_column_index):
    is_row_non_zero_list = [matrix[i,pivot_column_index].any() for i in range(matrix.shape[0])]

    # Ignore the rows before i
    for i in range(start_row_index):
        is_row_non_zero_list[i] = False

    # Ignore the current row even if it's non-zero
    is_row_non_zero_list[current_row_index] = False

    if np.any(is_row_non_zero_list):
        return np.argmax(is_row_non_zero_list)
    else:
        return -1

In [111]:
def highlight_cell(val):
    color = '#16a08544' if val==1.00 else ''

    return 'background-color:' + color

### def row_echelon_form()

In [112]:
def row_echelon_form(matrix, elements, debug=False):
    pivot_positions = []
    # For every row
    for i in range(matrix.shape[0]):
        left_most_non_zero_column_index = get_left_most_non_zero_column_index(matrix, start_row_index=i)
        if left_most_non_zero_column_index == -1:
            break

        # Step 1: try to put a 1 in the 'i'th row of the 'left_most_non_zero_column_index' column (aka pivot position)
        # check if its not non-zero already
        if matrix[i,left_most_non_zero_column_index] == 0:
            # find the top most non zero row below pivot
            top_most_non_zero_row_index = get_top_most_non_zero_row_index(
                matrix,
                start_row_index=i,
                pivot_column_index=left_most_non_zero_column_index)
            if top_most_non_zero_row_index == -1:
                break
            # add the found non-zero row to the 'i'th row (pivots row)
            matrix[i,:] += matrix[top_most_non_zero_row_index,:]
        # divide 'i'th row (pivots row) by the pivots value to get 1 in pivots position
        matrix[i,:] = matrix[i,:] / matrix[i, left_most_non_zero_column_index]

        # save the pivot positions for styling
        pivot_positions.append((i,left_most_non_zero_column_index))

        # Step 2: make all the other rows below the pivots row zero in the 'left_most_non_zero_column_index' column
        for row_index in range(i+1, matrix.shape[0]):
            # find the top most non zero row below pivot
            top_most_non_zero_row_index_thats_not_current_row = get_top_most_non_zero_row_index_thats_not_current_row(
                matrix,
                current_row_index=row_index,
                start_row_index=i,
                pivot_column_index=left_most_non_zero_column_index)
            
            if top_most_non_zero_row_index_thats_not_current_row == -1:
                break
            
            # add the found non-zero row to the current_row and make it zero in the 'left_most_non_zero_column_index'(pivot_index) column
            coef = -1.0 * matrix[row_index, left_most_non_zero_column_index] / matrix[top_most_non_zero_row_index_thats_not_current_row, left_most_non_zero_column_index]
            matrix[row_index,:] += coef * matrix[top_most_non_zero_row_index_thats_not_current_row,:]
        
        if debug:
            # show the coeff matrix as a dataframe
            print(f'i={i}')
            df = pd.DataFrame(
                matrix.round(2)+0, # +0 to get rid of -0.0.
                columns=["X"+str(x) for x in range(1,matrix.shape[1])]+["b"],
                index=elements
                )

            df = df.round(3)

            # for ii in pivot_positions:
            #     df.iloc[ii[0], ii[1]] = 1234
            
            # style = df.style.applymap(highlight_cell).copy()
            
            # for ii in pivot_positions:
            #     df.iloc[ii[0], ii[1]] = 1.00
            
            display(df)
    
    return matrix, pivot_positions

### def reduced_row_echelon_form()

In [113]:
def reduced_row_echelon_form(matrix, pivot_positions, elements, debug=False):
    # sort pivots from right to left
    pivot_positions = sorted(pivot_positions, key=lambda x: x[1], reverse=True)

    for pivot in pivot_positions:
        for current_row in range(pivot[0]):
            # add the pivot row to the current_row and make it zero in the pivot column
            coef = -1.0 * matrix[current_row, pivot[1]] / matrix[pivot[0], pivot[1]]
            matrix[current_row,:] += coef * matrix[pivot[0],:]
        
        if debug:
            print(f"Current pivot{pivot}")
            # show the coeff matrix as a dataframe
            df = pd.DataFrame(
                matrix.round(2)+0, # +0 to get rid of -0.0.
                columns=["X"+str(x) for x in range(1,matrix.shape[1])]+["b"],
                index=elements
                )
            df = df.round(3)
            # df = df.style.applymap(highlight_cell)
            display(df)
            print()
    

## Main

### Decoding the input file

In [114]:
# The input file should contain 2 lines
# first line shows all of the chemical elements seperated by a space example: 'C H O'
# second line shows the chemical reaction example: 'C2 H6 + O2 -> C O2 + H2 O'
with open('input.txt', 'r') as f:
    lines = f.readlines()

    if len(lines)<2:
        print('Error: File is empty!')
    else:
        elements = lines[0].split()

        # Split by ->
        compounds_left = lines[1].split('->')[0].split('+')
        compounds_right = lines[1].split('->')[1].split('+')
        
        # Get M, N
        M = len(elements)
        N = len(compounds_left) + len(compounds_right) + 1 # +1 for b=0

        # Coeff matrix
        coeff_matrix = np.zeros((M, N), dtype=np.float64)

        # Extract compounds
        compounds_left = [x.split() for x in compounds_left]
        compounds_right = [x.split() for x in compounds_right]

        i = 0
        # Loop through the left hand side compounds and add them to the coeff matrix as column vectors
        for compound in compounds_left:
            coeff_matrix[:,i] = compound_to_vector(compound, elements)
            i += 1

        # Loop through the right hand side compounds and add them to the coeff matrix as column vectors but multiplied by -1
        for compound in compounds_right:
            coeff_matrix[:,i] = -1*compound_to_vector(compound, elements)
            i += 1

print('Number of elements: ', M)
print('Number of compounds: ', N-1)
print('Elements: ', elements)
print('compounds LHS: ', compounds_left)
print('compounds RHS: ', compounds_right)
print('\nCoefficients Matrix: ')

# show the coeff matrix as a dataframe
df = pd.DataFrame(
    coeff_matrix, 
    columns=["X"+str(x) for x in range(1,N)]+["b"],
    index=elements
    )
display(df)

Number of elements:  3
Number of compounds:  4
Elements:  ['C', 'H', 'O']
compounds LHS:  [['C2', 'H6'], ['O2']]
compounds RHS:  [['C', 'O2'], ['H2', 'O']]

Coefficients Matrix: 


,X1,X2,X3,X4,b
C,2.0,0.0,-1.0,-0.0,0.0
H,6.0,0.0,-0.0,-2.0,0.0
O,0.0,2.0,-2.0,-1.0,0.0


### Echelon form
Convert the Augmented Coefficients Matrix into the echelon form.

In [115]:
# test matrix
# matrix = np.array(
#     [
#         [0, 3 ,-6, 6, 4, -5,],
#         [3, -7 ,8, -5, 8, 9,],
#         [3, -9 ,12, -9, 6, 15,],
#     ], dtype=np.float64)

matrix = coeff_matrix

In [116]:
print("Converting The Augmented Matrix to Row Echelon Form\n")
row_echelon_matrix, pivot_positions = row_echelon_form(matrix, elements, debug=True)
print(f"Pivot positions: {', '.join(list(map(str,pivot_positions)))}")

Converting The Augmented Matrix to Row Echelon Form

i=0


,X1,X2,X3,X4,b
C,1.0,0.0,-0.5,0.0,0.0
H,0.0,0.0,3.0,-2.0,0.0
O,0.0,2.0,-2.0,-1.0,0.0


i=1


,X1,X2,X3,X4,b
C,1.0,0.0,-0.5,0.0,0.0
H,0.0,1.0,0.5,-1.5,0.0
O,0.0,0.0,-3.0,2.0,0.0


i=2


,X1,X2,X3,X4,b
C,1.0,0.0,-0.5,0.00,0.0
H,0.0,1.0,0.5,-1.50,0.0
O,0.0,0.0,1.0,-0.67,0.0


Pivot positions: (0, 0), (1, 1), (2, 2)


### Reduced Echelon form
Convert echelon form to reduced echelon form.

In [117]:
print("Converting The Augmented Matrix to Reduced Row Echelon Form\n")
reduced_row_echelon_matrix = reduced_row_echelon_form(row_echelon_matrix, pivot_positions, elements, debug=True)

Converting The Augmented Matrix to Reduced Row Echelon Form

Current pivot(2, 2)


,X1,X2,X3,X4,b
C,1.0,0.0,0.0,-0.33,0.0
H,0.0,1.0,0.0,-1.17,0.0
O,0.0,0.0,1.0,-0.67,0.0



Current pivot(1, 1)


,X1,X2,X3,X4,b
C,1.0,0.0,0.0,-0.33,0.0
H,0.0,1.0,0.0,-1.17,0.0
O,0.0,0.0,1.0,-0.67,0.0



Current pivot(0, 0)


,X1,X2,X3,X4,b
C,1.0,0.0,0.0,-0.33,0.0
H,0.0,1.0,0.0,-1.17,0.0
O,0.0,0.0,1.0,-0.67,0.0


### Solve for values of X = [X1, X2, ...]

In [118]:
basic_variables = [i[1] for i in pivot_positions]
print(f"Basic variables: {', '.join(list(map(lambda x: 'X'+str(x+1), basic_variables)))}")

free_variables = [i for i in range(1,matrix.shape[1]-1) if i not in basic_variables]
print(f"Free variables: {', '.join(list(map(lambda x: 'X'+str(x+1), free_variables)))}")

Basic variables: X1, X2, X3
Free variables: X4


In [119]:
# Fix the free variables as 1
result = np.zeros(matrix.shape[1]-1)
result[free_variables] = 1

# Find the basic variables 
for pivot in pivot_positions:
    result[pivot[1]] = matrix[pivot[0], free_variables]@(-1*result[free_variables]) # eg. (X3, X4) dot (-2, 3) and *-1 because we moved them to RHS
    result[pivot[1]] += matrix[pivot[0], -1] # +b

result_df = pd.DataFrame(result.reshape(1, -1), columns=["X"+str(x) for x in range(1,result.shape[0]+1)])
display(result_df)

,X1,X2,X3,X4
0,0.333333,1.166667,0.666667,1.0


### Bonus: Find the smallest integer multiple

### Result

#### Floating point coefficients

In [120]:
result_text = ""

# Add the left hand side to the output text
compound_id = 0
for idx, compound in enumerate(compounds_left):
    if idx != 0:
        result_text += " + "
    result_text += f"({round(result[compound_id], 3)}) " + "".join(compound)
    compound_id += 1

# Add -> to the output text
result_text += " -> "

# Add the right hand side to the output text
for idx, compound in enumerate(compounds_right):
    if idx != 0:
        result_text += " + "
    result_text += f"({round(result[compound_id], 3)}) " + "".join(compound)
    compound_id += 1
    
print(result_text)

(0.333) C2H6 + (1.167) O2 -> (0.667) CO2 + (1.0) H2O


#### Fraction coefficients

In [121]:
Fraction(0.3333333333333333333).limit_denominator()

Fraction(1, 3)

In [122]:
fraction_result = list(map(lambda x: Fraction(x).limit_denominator(), result))
denominators_list = list(map(lambda x: x.denominator, fraction_result))
print("fraction_result:", ", ".join([f"{i.numerator}/{i.denominator}" for i in fraction_result]))
print("denominators_list:", denominators_list)

fraction_result: 1/3, 7/6, 2/3, 1/1
denominators_list: [3, 6, 3, 1]


In [123]:
result_text = ""

# Add the left hand side to the output text
compound_id = 0
for idx, compound in enumerate(compounds_left):
    if idx != 0:
        result_text += " + "
    result_text += f"({fraction_result[compound_id].numerator}/{fraction_result[compound_id].denominator}) " + "".join(compound)
    compound_id += 1

# Add -> to the output text
result_text += " -> "

# Add the right hand side to the output text
for idx, compound in enumerate(compounds_right):
    if idx != 0:
        result_text += " + "
    result_text += f"({fraction_result[compound_id].numerator}/{fraction_result[compound_id].denominator}) " + "".join(compound)
    compound_id += 1
    
print(result_text)

(1/3) C2H6 + (7/6) O2 -> (2/3) CO2 + (1/1) H2O


#### Integer coefficients

In [124]:
least_common_multiplier = reduce(lcm, denominators_list)
new_result = result * least_common_multiplier
print("Integer coefficients:", new_result)

Integer coefficients: [2. 7. 4. 6.]


In [125]:
result_text = ""

# Add the left hand side to the output text
compound_id = 0
for idx, compound in enumerate(compounds_left):
    if idx != 0:
        result_text += " + "
    result_text += f"({int(new_result[compound_id])}) " + "".join(compound)
    compound_id += 1

# Add -> to the output text
result_text += " -> "

# Add the right hand side to the output text
for idx, compound in enumerate(compounds_right):
    if idx != 0:
        result_text += " + "
    result_text += f"({int(new_result[compound_id])}) " + "".join(compound)
    compound_id += 1
    
print(result_text)

(2) C2H6 + (7) O2 -> (4) CO2 + (6) H2O
